In [ ]:
!pip install fastapi # Установим FastAPI
!pip install uvicorn # Установим ASGI-сервер
!pip install python-multipart # Необходимая зависимость для FastAPI (для работы с данными отправленных форм на сайте)

In [ ]:
%%writefile main.py

import uuid # генератор случайного токена, как id пользователя
from fastapi import FastAPI, Body
from fastapi.responses import JSONResponse

# Определяем класс Person с полями:
#  name - имя
#  age  - возраст
#  id   - идентификатор пользователя
class PC:
    def __init__(self, typePC, vendor, model, cost, description):
        self.typePC = typePC
        self.vendor = vendor
        self.model = model
        self.cost = cost
        self.description = description
        self.id = str(uuid.uuid4())
# условная база данных - набор объектов
PC_DB = [PC("CPU","Intel", "i5-12400", 12199, "LGA 1700, 6 x 2.5 ГГц, L2 - 7.5 МБ, L3 - 18 МБ, 2 х DDR4, DDR5-4800 МГц, TDP 117 Вт"),
         PC("CPU","AMD", "Ryzen 5 7500F", 17499, "AM5, 6 x 3.7 ГГц, L2 - 6 МБ, L3 - 32 МБ, 2 х DDR5-5200 МГц, TDP 65 Вт"),
         PC("CPU","Intel", "i7-12700KF", 28799, "LGA 1700, 8P x 3.6 ГГц, 4E x 2.7 ГГц, L2 - 12 МБ, L3 - 25 МБ, 2 х DDR4, DDR5-4800 МГц, TDP 190 Вт"),
         PC("GPU","MSI", "RTX 4060 VENTUS 2X", 40999, "PCIe 4.0 8 ГБ GDDR6, 128 бит, 3 x DisplayPort, HDMI, GPU 1830 МГц"),
         PC("GPU","GIGABYTE", "RTX 4070 SUPER", 89999, "PCIe 4.0 12 ГБ GDDR6X, 192 бит, 3 x DisplayPort, HDMI, GPU 1980 МГц"),
         PC("GPU","ASUS", "RTX 4070 SUPER TUF Gaming", 106999, "PCIe 4.0 12 ГБ GDDR6X, 192 бит, 3 x DisplayPort, HDMI, GPU 1980 МГц"),
         PC("DDR4","Kingston", "FURY Beast Black", 8899, "DDR4, 16 ГБx2 шт, 3200 МГц, 16(CL)-20-20"),
         PC("DDR5","ADATA", "XPG Lancer Blade RGB", 13499, "DDR5, 16 ГБx2 шт, 6000 МГц, 30(CL)-40-40"),
         PC("DDR5","Corsair", "Vengeance", 17999, "DDR5, 16 ГБx2 шт, 6000 МГц, 36(CL)-38-38-76"),]




# для поиска пользователя в списке
def find_PC(id):
   for pc in PC_DB:
        if pc.id == id:
           return pc
   return None

app = FastAPI()



@app.get("/api/pc",
    tags=["list pc items"],
    summary="Список комплектующих",
    description="Получение списка комплектующих из базы",
    )
async def get_pc():
    return PC_DB

@app.get("/api/pc/total",
    tags=["total cost"],
    summary="Сумма",
    description="Общая стоимость"
    )
async def get_pc_total():
    total_cost = sum(pc.cost for pc in PC_DB)
    return JSONResponse(
                content={ "message": f"Общая стоимость всех элементов в базе данных: {total_cost} руб." }
    )

@app.get("/api/pc/{id}",
    tags=["get unit from pc items"],
    summary="Комплектующие",
    description="Получение комплектующих по идентификатору")
async def get_pc(id):
    # получаем комплектующие по id
    pcOne = find_PC(id)
    print(pcOne)
    # если не найден, отправляем статусный код и сообщение об ошибке
    if pcOne==None:
        return JSONResponse(
                status_code=404,
                content={ "message": "Комплектующие не найдены" }
        )
    #если комплектующее найдено, отправляем его
    return pcOne

@app.post("/api/pc",
    tags=["create new unit for pc items"],
    summary="Создание комплектующих",
    description="Добавление новых комплектующих в базу данных")
async def create_pc(data  = Body()):
    pcOne = PC(data["typePC"], data["vendor"], data["model"], data["cost"], data["description"])
    # добавляем объект в список
    PC_DB.append(pcOne)
    return pcOne

Overwriting main.py


In [ ]:
!nohup uvicorn main:app --reload &

nohup: appending output to 'nohup.out'


In [ ]:
!cat nohup.out

INFO:     Will watch for changes in these directories: ['/content']
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [1143] using StatReload
INFO:     Started server process [1146]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     127.0.0.1:33484 - "GET /test HTTP/1.1" 200 OK
INFO:     77.40.27.54:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     77.40.27.54:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     77.40.27.54:0 - "GET /test HTTP/1.1" 200 OK
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [1146]
INFO:     Stopping reloader process [1143]
INFO:     Will watch for changes in these directories: ['/content']
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [1726] using StatReload
INFO:     Started server process [1728]
INFO:  

**Находим стоимость всех элементов в ДБ**

In [ ]:
r = requests.get('http://127.0.0.1:8000/api/pc/total')
print(r.text)

{"message":"Общая стоимость всех элементов в базе данных: 336891 руб."}


**Выводим все элементы ДБ**

In [ ]:
import requests
r = requests.get('http://127.0.0.1:8000/api/pc')
pcs = r.json()
for pc in pcs:
  print('Комплектующие', pc['typePC'], '-', pc['vendor'], '-', pc['model'], '-', pc['cost'], '-', pc['description'], '(id =', pc['id'], ')' )

Комплектующие CPU - Intel - i5-12400 - 12199 - LGA 1700, 6 x 2.5 ГГц, L2 - 7.5 МБ, L3 - 18 МБ, 2 х DDR4, DDR5-4800 МГц, TDP 117 Вт (id = 46866ba3-6880-4848-8e83-5d290b5a9db9 )
Комплектующие CPU - AMD - Ryzen 5 7500F - 17499 - AM5, 6 x 3.7 ГГц, L2 - 6 МБ, L3 - 32 МБ, 2 х DDR5-5200 МГц, TDP 65 Вт (id = b58c06e8-28fc-45c5-9a61-b8f434b0f4a1 )
Комплектующие CPU - Intel - i7-12700KF - 28799 - LGA 1700, 8P x 3.6 ГГц, 4E x 2.7 ГГц, L2 - 12 МБ, L3 - 25 МБ, 2 х DDR4, DDR5-4800 МГц, TDP 190 Вт (id = 88856a8b-9630-470e-b356-9e574dd5bb31 )
Комплектующие GPU - MSI - RTX 4060 VENTUS 2X - 40999 - PCIe 4.0 8 ГБ GDDR6, 128 бит, 3 x DisplayPort, HDMI, GPU 1830 МГц (id = 4ff5c4a9-fca4-4fce-afd0-8cf8871923fa )
Комплектующие GPU - GIGABYTE - RTX 4070 SUPER - 89999 - PCIe 4.0 12 ГБ GDDR6X, 192 бит, 3 x DisplayPort, HDMI, GPU 1980 МГц (id = ded497f2-4714-454c-bb2c-72fe8d87c1b2 )
Комплектующие GPU - ASUS - RTX 4070 SUPER TUF Gaming - 106999 - PCIe 4.0 12 ГБ GDDR6X, 192 бит, 3 x DisplayPort, HDMI, GPU 1980 МГц 

In [ ]:
pcs

[{'typePC': 'CPU',
  'vendor': 'Intel',
  'model': 'i5-12400',
  'cost': 12199,
  'description': 'LGA 1700, 6 x 2.5 ГГц, L2 - 7.5 МБ, L3 - 18 МБ, 2 х DDR4, DDR5-4800 МГц, TDP 117 Вт',
  'id': '46866ba3-6880-4848-8e83-5d290b5a9db9'},
 {'typePC': 'CPU',
  'vendor': 'AMD',
  'model': 'Ryzen 5 7500F',
  'cost': 17499,
  'description': 'AM5, 6 x 3.7 ГГц, L2 - 6 МБ, L3 - 32 МБ, 2 х DDR5-5200 МГц, TDP 65 Вт',
  'id': 'b58c06e8-28fc-45c5-9a61-b8f434b0f4a1'},
 {'typePC': 'CPU',
  'vendor': 'Intel',
  'model': 'i7-12700KF',
  'cost': 28799,
  'description': 'LGA 1700, 8P x 3.6 ГГц, 4E x 2.7 ГГц, L2 - 12 МБ, L3 - 25 МБ, 2 х DDR4, DDR5-4800 МГц, TDP 190 Вт',
  'id': '88856a8b-9630-470e-b356-9e574dd5bb31'},
 {'typePC': 'GPU',
  'vendor': 'MSI',
  'model': 'RTX 4060 VENTUS 2X',
  'cost': 40999,
  'description': 'PCIe 4.0 8 ГБ GDDR6, 128 бит, 3 x DisplayPort, HDMI, GPU 1830 МГц',
  'id': '4ff5c4a9-fca4-4fce-afd0-8cf8871923fa'},
 {'typePC': 'GPU',
  'vendor': 'GIGABYTE',
  'model': 'RTX 4070 SUPER',
 

**Добавляем новый элемент в ДБ**

In [ ]:
typePC = input('Введи тип:')
vendor = input('Введи вендора:')
model = input('Введи модель:')
cost = input('Введи стоимость:')
description = input('Введи Описание:')
new_pc = {'typePC': typePC, 'vendor': vendor, 'model' : model, 'cost' : cost, 'description' :  description}

r = requests.post('http://127.0.0.1:8000/api/pc', json = new_pc)
pc = r.json()
print('Добавлено комплектующее', pc['typePC'], '-', pc['vendor'], '-', pc['model'], '-', pc['cost'], '-', pc['description'], '(id =', pc['id'], ')' )

Введи тип:CPU
Введи вендора:Intel
Введи модель:i5 13600k
Введи стоимость:32599
Введи Описание:none
Добавлено комплектующее CPU - Intel - i5 13600k - 32599 - none (id = a4f6d666-8ac7-4062-858a-4883c8dc6b8f )


**Поиск по id и вывод стоимости**

In [ ]:
r = requests.get('http://127.0.0.1:8000/api/pc/'+pcs[3]['id'])
# Проверка статуса ответа
if r.status_code != 200:
    print(f"Ошибка: Сервер вернул статус код {r.status_code}")
    print("Ответ сервера:", r.text)
else:
    try:
        pc = r.json()
        print('URI для отправки запроса:', r.request.url)
        print('Комплектующие', pc['typePC'], '-', pc['vendor'], '-', pc['model'], '- стоимость', pc['cost'], 'рублей')
        all = pc['cost']
    except ValueError:
        print("Ошибка: Не удалось декодировать JSON. Ответ сервера:", r.text)
print(type(all))

URI для отправки запроса: http://127.0.0.1:8000/api/pc/4ff5c4a9-fca4-4fce-afd0-8cf8871923fa
Комплектующие GPU - MSI - RTX 4060 VENTUS 2X - стоимость 40999 рублей
<class 'int'>


In [ ]:
!ssh -o "StrictHostKeyChecking no" -R 80:localhost:8000 serveo.net

Forwarding HTTP traffic from https://bf430152fad1f8cc285d975a604bce86.serveo.net
HTTP request from 150.241.94.30 to https://bf430152fad1f8cc285d975a604bce86.serveo.net/
HTTP request from 150.241.94.30 to https://bf430152fad1f8cc285d975a604bce86.serveo.net/favicon.ico
HTTP request from 150.241.94.30 to https://bf430152fad1f8cc285d975a604bce86.serveo.net/docs
HTTP request from 150.241.94.30 to https://bf430152fad1f8cc285d975a604bce86.serveo.net/openapi.json
